# Part 3: AI Paper Review Agent 활용하기

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. 기존 오픈소스 **Paper Review Agent** 도구들 알아보기
2. **agentic-paper-review** 설치 및 실행
3. 본인 논문으로 AI 리뷰 받아보기

---

## 기존 오픈소스 도구들

| 도구 | 특징 | GitHub |
|------|------|--------|
| **agentic-paper-review** | LangGraph 9노드, ρ=0.74 | [debashis1983/agentic-paper-review](https://github.com/debashis1983/agentic-paper-review) |
| **AgentReview** | 다중 에이전트, 리뷰어 성격 시뮬레이션 | [Ahren09/AgentReview](https://github.com/Ahren09/AgentReview) |
| **AI-Scientist** | 전체 연구 파이프라인 | [SakanaAI/AI-Scientist](https://github.com/SakanaAI/AI-Scientist) |

이번 실습에서는 **agentic-paper-review**를 사용합니다.

---
## Part 1: Setup

### API Key 발급 링크

| API | 링크 | 비고 |
|-----|------|------|
| **OpenAI** (필수) | https://platform.openai.com/api-keys | agentic-paper-review에 필요 |
| **Gemini** (대안) | https://aistudio.google.com/apikey | 무료, 넉넉한 quota |

> ⚠️ **agentic-paper-review**는 OpenAI API가 필요합니다.

In [ ]:
# Cell 1: 환경 설정
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/Hands-on-3/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: agentic-paper-review 설치
import os
import urllib.request
import shutil

AGENT_DIR = os.path.join(WORKSHOP_DIR, "agentic-paper-review")

# Colab: 기존 폴더 삭제 후 새로 설치 (최신 버전 보장)
if IN_COLAB and os.path.exists(AGENT_DIR):
    print("🗑️ 기존 agentic-paper-review 삭제 중...")
    shutil.rmtree(AGENT_DIR)

if not os.path.exists(AGENT_DIR):
    print("📥 agentic-paper-review 클론 중...")
    !git clone https://github.com/debashis1983/agentic-paper-review.git {AGENT_DIR}
    print("✅ 클론 완료!")
else:
    print(f"✅ 이미 설치됨: {AGENT_DIR}")

# Gemini 지원 agent.py로 교체 (워크샵용 수정 버전)
print("\n🔄 Gemini 지원 버전으로 업데이트 중...")
agent_url = "https://raw.githubusercontent.com/yoonlee78/2026_SNUIPS_AI_Workshop/master/Hands-on-3/lib/agent_gemini.py"
agent_path = os.path.join(AGENT_DIR, "agent.py")
try:
    urllib.request.urlretrieve(agent_url, agent_path)
    print("✅ Gemini 지원 agent.py 적용 완료!")
except Exception as e:
    print(f"⚠️ agent.py 업데이트 실패: {e}")
    print("   (원본 버전 사용 - OpenAI API 필요)")

# 의존성 설치 (langchain-google-genai 추가 - Gemini API용)
print("\n📦 의존성 설치 중...")
!pip install langgraph langchain langchain-openai langchain-anthropic langchain-google-genai pdfplumber tavily-python pymupdf -q
print("✅ 의존성 설치 완료!")

In [ ]:
# Cell 3: API Key 로딩
import os

# ✏️ 사용할 API 선택 (아래 중 하나만 주석 해제)
# USE_API = "openai"   # OpenAI 사용 (agentic-paper-review 기본)
USE_API = "gemini"   # Gemini 사용 (무료)
# USE_API = "auto"       # 자동 선택 (OpenAI 우선)

# ─────────────────────────────────────────
# API Key 로딩
if IN_COLAB:
    from google.colab import userdata
    
    # Gemini Key (여러 이름 시도)
    GEMINI_API_KEY = None
    for key_name in ['GEMINI_API_KEY', 'gemini key', 'gemini_key', 'GOOGLE_API_KEY']:
        try:
            key = userdata.get(key_name)
            if key:
                GEMINI_API_KEY = key
                break
        except:
            pass
    
    # OpenAI Key (여러 이름 시도)
    OPENAI_API_KEY = None
    for key_name in ['OPENAI_API_KEY', 'openai key', 'openai_key']:
        try:
            key = userdata.get(key_name)
            if key:
                OPENAI_API_KEY = key
                break
        except:
            pass
else:
    from dotenv import load_dotenv
    load_dotenv(override=True)
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# ─────────────────────────────────────────
# 모든 키를 os.environ에 설정 (agentic-paper-review가 내부적으로 필요)
if OPENAI_API_KEY and not OPENAI_API_KEY.startswith('your_'):
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
if GEMINI_API_KEY and not GEMINI_API_KEY.startswith('your_'):
    os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
    os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

# ─────────────────────────────────────────
# API 선택 로직 (표시용)
API_PROVIDER = None

if USE_API == "openai":
    if OPENAI_API_KEY and not OPENAI_API_KEY.startswith('your_'):
        API_PROVIDER = 'openai'
        print("✅ OpenAI API 사용")
    else:
        print("❌ OpenAI API Key가 없습니다")

elif USE_API == "gemini":
    if GEMINI_API_KEY and not GEMINI_API_KEY.startswith('your_'):
        API_PROVIDER = 'gemini'
        print("✅ Gemini API 사용")
    else:
        print("❌ Gemini API Key가 없습니다")
        if IN_COLAB:
            print("   Colab: 좌측 🔑 > GEMINI_API_KEY 또는 'gemini key' 추가")
        else:
            print("   로컬: .env 파일에 GEMINI_API_KEY 추가")

else:  # auto
    if OPENAI_API_KEY and not OPENAI_API_KEY.startswith('your_'):
        API_PROVIDER = 'openai'
        print("✅ OpenAI API 사용 (자동)")
    elif GEMINI_API_KEY and not GEMINI_API_KEY.startswith('your_'):
        API_PROVIDER = 'gemini'
        print("✅ Gemini API 사용 (자동)")

if API_PROVIDER is None:
    print("❌ API Key가 필요합니다!")
    if IN_COLAB:
        print("   Colab: 좌측 🔑 아이콘 > Secrets에 키 추가")
    else:
        print("   로컬: .env 파일에 키 추가")

---
## Part 2: agentic-paper-review 소개

### 아키텍처

**9-Node LangGraph Workflow:**
```
[1. PDF→MD] → [2. 메타데이터] → [3. 쿼리생성] → [4. 웹검색]
      ↓
[5. 관련성평가] → [6. 리플렉션] → [7. 요약] → [8. 리뷰생성] → [9. 점수산정]
```

> 💡 **MD/TXT 파일**: 1번 노드 스킵 → 바로 2번부터 시작
> 💡 **DOCX 파일**: 텍스트 추출 후 2번부터 시작

### 평가 차원
- **Soundness**: 방법론의 엄밀성
- **Presentation**: 논문의 명확성
- **Contribution**: 기여도

### 성능
- Spearman ρ = 0.74 (46,748개 실제 리뷰로 학습)

In [ ]:
# Cell 4: 논문 로딩 (PDF, MD, TXT, DOCX 모두 지원)
import os

# ✏️ 본인 논문 경로 입력 (my_manuscript.md가 있으면 자동 사용)
MY_PAPER_PATH = "input/my_manuscript.md"  # 또는 다른 파일 경로

# ─────────────────────────────────────────
PAPER_PATH = None
PAPER_TEXT = None  # MD/TXT/DOCX인 경우 텍스트 저장

# 1순위: 본인 논문
if MY_PAPER_PATH and os.path.exists(MY_PAPER_PATH):
    PAPER_PATH = MY_PAPER_PATH
    ext = os.path.splitext(MY_PAPER_PATH)[1].lower()

    if ext == '.pdf':
        print(f"✅ PDF 논문: {MY_PAPER_PATH}")
    elif ext in ['.md', '.txt']:
        with open(MY_PAPER_PATH, 'r', encoding='utf-8') as f:
            PAPER_TEXT = f.read()
        print(f"✅ 텍스트 논문: {MY_PAPER_PATH} (1번 노드 스킵)")
    elif ext in ['.docx', '.doc']:
        try:
            import docx
            doc = docx.Document(MY_PAPER_PATH)
            PAPER_TEXT = "\n".join([para.text for para in doc.paragraphs])
            print(f"✅ Word 논문: {MY_PAPER_PATH} (1번 노드 스킵)")
        except ImportError:
            print("⚠️ python-docx 필요: pip install python-docx")

# 2순위: 샘플 논문
if PAPER_PATH is None:
    SAMPLE_PATH = os.path.join(WORKSHOP_DIR, "input", "sample_manuscript.md")

    # Colab: 샘플 파일이 없으면 GitHub에서 다운로드
    if not os.path.exists(SAMPLE_PATH):
        os.makedirs(os.path.dirname(SAMPLE_PATH), exist_ok=True)
        print("📥 샘플 논문 다운로드 중...")
        import urllib.request
        url = "https://raw.githubusercontent.com/yoonlee78/2026_SNUIPS_AI_Workshop/master/Hands-on-3/input/sample_manuscript.md"
        urllib.request.urlretrieve(url, SAMPLE_PATH)
        print("✅ 다운로드 완료!")

    if os.path.exists(SAMPLE_PATH):
        PAPER_PATH = SAMPLE_PATH
        with open(SAMPLE_PATH, 'r', encoding='utf-8') as f:
            PAPER_TEXT = f.read()
        print(f"📄 샘플 논문 사용 중 (1번 노드 스킵)")
        print("")
        print("="*50)
        print("⚠️ 본인 논문으로 실습하려면:")
        print("   MY_PAPER_PATH에 파일 경로를 입력하세요")
        print("   지원 형식: PDF, MD, TXT, DOCX")
        print("="*50)

if PAPER_PATH:
    print(f"\n📄 논문 경로: {PAPER_PATH}")
    if PAPER_TEXT:
        print(f"   텍스트 길이: {len(PAPER_TEXT)} 문자")
else:
    print("❌ 논문을 찾을 수 없습니다")
    print("   MY_PAPER_PATH에 본인 논문 경로를 입력하세요")

---
## Part 3: agentic-paper-review 실행

In [ ]:
# Cell 5: agentic-paper-review 모듈 로딩
import sys

# Colab: 기존에 로드된 모듈 제거 (새 버전 강제 적용)
if IN_COLAB:
    modules_to_remove = [key for key in sys.modules.keys() if 'agent' in key.lower()]
    for mod in modules_to_remove:
        del sys.modules[mod]
    print("🔄 Colab: 모듈 캐시 초기화")

# 경로 추가 (이미 있으면 제외)
if AGENT_DIR not in sys.path:
    sys.path.insert(0, AGENT_DIR)

# API Key 설정 확인 (모듈 import 전)
print(f"\n🔑 API Key 상태:")
print(f"   GOOGLE_API_KEY: {'✅ 설정됨' if os.getenv('GOOGLE_API_KEY') else '❌ 없음'}")
print(f"   OPENAI_API_KEY: {'✅ 설정됨' if os.getenv('OPENAI_API_KEY') else '❌ 없음'}")

try:
    from agent import AgenticPaperReviewer
    print("\n✅ AgenticPaperReviewer 로딩 완료")
except ImportError as e:
    print(f"\n⚠️ 모듈 로딩 실패: {e}")
    print("   Cell 2를 먼저 실행하세요 (설치)")
except Exception as e:
    print(f"\n❌ 예상치 못한 오류: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Cell 6: 리뷰 실행
# agentic-paper-review는 paper_content 파라미터로 텍스트를 받으면
# 자동으로 pdf_to_markdown 노드를 스킵합니다!

if PAPER_PATH:
    print(f"🔄 논문 리뷰 시작: {PAPER_PATH}")
    if PAPER_TEXT:
        print("   (텍스트 입력 → 1번 노드 스킵)")
    print("   (1-2분 소요)")
    print("="*60)
    
    # 리뷰 실행 전 API 키 최종 확인
    print(f"\n📡 API 설정 최종 확인:")
    print(f"   GOOGLE_API_KEY: {os.getenv('GOOGLE_API_KEY', '')[:15]}..." if os.getenv('GOOGLE_API_KEY') else "   GOOGLE_API_KEY: ❌ 없음")
    print("")
    
    # 리뷰 실행
    reviewer = AgenticPaperReviewer()
    
    try:
        if PAPER_TEXT:
            # 텍스트가 있으면 paper_content로 전달 (1번 노드 스킵)
            review_result = await reviewer.review_paper(
                paper_content=PAPER_TEXT,
                target_venue="Psychology"
            )
        else:
            # PDF면 paper_path로 전달 (전체 워크플로우)
            review_result = await reviewer.review_paper(
                paper_path=PAPER_PATH,
                target_venue="Psychology"
            )
        
        # 결과 상태 확인
        status = review_result.get('status', 'unknown')
        errors = review_result.get('metadata', {}).get('errors', [])
        
        if status == 'complete' and review_result.get('review'):
            print("\n✅ 리뷰 완료!")
        elif status == 'failed' or errors:
            print(f"\n⚠️ 리뷰 실패 또는 불완전")
            print(f"   상태: {status}")
            if errors:
                print(f"   에러: {errors}")
        else:
            print(f"\n⚠️ 예상치 못한 결과 (상태: {status})")
            
    except Exception as e:
        print(f"\n❌ 리뷰 실행 중 오류: {e}")
        import traceback
        traceback.print_exc()
        review_result = None
else:
    print("❌ 논문 경로를 먼저 설정하세요 (Cell 4)")

In [ ]:
# Cell 7: 결과 확인
if 'review_result' in dir() and review_result:
    print("📊 리뷰 결과")
    print("="*60)
    
    # 상태 확인
    print(f"\n상태: {review_result.get('status', 'unknown')}")
    
    # 논문 정보
    metadata = review_result.get('paper_metadata')
    if metadata:
        print(f"\n📄 논문: {metadata.get('title', 'Unknown')}")
    
    # 점수 출력
    scores = review_result.get('scores', {})
    if scores.get('dimensions'):
        print("\n📈 점수:")
        for dim in scores['dimensions']:
            print(f"  {dim['name']}: {dim['score']} - {dim['justification'][:50]}...")
    
    if scores.get('final_score'):
        print(f"\n🎯 최종 점수: {scores['final_score_display']}")
    
    # 리뷰 출력
    review = review_result.get('review', '')
    if review:
        print("\n📋 리뷰:")
        print("="*60)
        print(review[:3000])  # 처음 3000자만
        if len(review) > 3000:
            print(f"\n... ({len(review)}자 중 3000자만 표시)")
    
    # 관련 연구
    related = review_result.get('related_works', [])
    if related:
        print(f"\n📚 관련 연구 ({len(related)}개):")
        for w in related[:5]:
            print(f"  - {w['title'][:50]}... (관련도: {w['relevance']:.2f})")
    
    # 에러 확인
    errors = review_result.get('metadata', {}).get('errors', [])
    if errors:
        print(f"\n⚠️ 에러: {errors}")
    
    print("="*60)
else:
    print("⚠️ Cell 6을 먼저 실행하세요")

In [ ]:
# Cell 7-1: 리뷰 한글 번역 (선택)
# ============================================================
# True로 설정하면 리뷰를 한글로 번역합니다 (Gemini API 사용)
# ============================================================

TRANSLATE_TO_KOREAN = False  # True로 변경하면 번역 실행

# ============================================================

if TRANSLATE_TO_KOREAN:
    if 'review_result' not in dir() or not review_result:
        print("⚠️ Cell 6을 먼저 실행하세요 (리뷰 생성)")
    elif not GEMINI_API_KEY:
        print("⚠️ Gemini API Key가 필요합니다 (번역용)")
    else:
        import google.generativeai as genai
        
        review_text = review_result.get('review', '')
        if not review_text:
            print("⚠️ 번역할 리뷰가 없습니다")
        else:
            print("🔄 리뷰 번역 중... (Gemini)")
            
            genai.configure(api_key=GEMINI_API_KEY)
            model = genai.GenerativeModel('gemini-2.5-flash')
            
            # 간결한 번역 프롬프트
            prompt = f"""다음 영어 논문 리뷰를 한국어로 번역하세요. 
학술적 어투를 유지하고, 전문 용어는 영어를 병기하세요.

---
{review_text}
---

한국어 번역:"""
            
            response = model.generate_content(prompt)
            review_korean = response.text
            
            # 결과 저장
            review_result['review_korean'] = review_korean
            
            print("✅ 번역 완료!")
            print("="*60)
            print(review_korean[:2000])
            if len(review_korean) > 2000:
                print(f"\n... ({len(review_korean)}자 중 2000자만 표시)")
            print("="*60)
else:
    print("ℹ️ 한글 번역을 원하면 TRANSLATE_TO_KOREAN = True 로 설정하세요")

In [ ]:
# Cell 8: 결과 저장 (JSON + Markdown)
import json
import os
from datetime import datetime

if 'review_result' in dir() and review_result:
    output_dir = os.path.join(WORKSHOP_DIR, "outputs", "3_agent_result")
    os.makedirs(output_dir, exist_ok=True)
    
    # 타임스탬프 생성
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # review_result는 딕셔너리임
    result_dict = {
        "method": "agentic-paper-review",
        "paper_path": PAPER_PATH,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "status": review_result.get('status', 'unknown'),
        "paper_metadata": review_result.get('paper_metadata', {}),
        "review": review_result.get('review', ''),
        "scores": review_result.get('scores', {}),
        "related_works": review_result.get('related_works', []),
        "errors": review_result.get('metadata', {}).get('errors', [])
    }
    
    # 1. JSON 저장 (타임스탬프 포함)
    json_path = os.path.join(output_dir, f"review_{timestamp}.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(result_dict, f, ensure_ascii=False, indent=2)
    
    # 2. Markdown 저장 (타임스탬프 포함)
    md_path = os.path.join(output_dir, f"review_{timestamp}.md")
    
    scores = result_dict.get('scores', {})
    dimensions = scores.get('dimensions', [])
    
    md_content = f"""# AI Paper Review (agentic-paper-review)

**생성 시간**: {result_dict['timestamp']}

## 논문 정보
- **제목**: {result_dict.get('paper_metadata', {}).get('title', 'Unknown')}
- **상태**: {result_dict.get('status', 'unknown')}
- **최종 점수**: {scores.get('final_score_display', 'N/A')}

## 점수 상세
"""
    for dim in dimensions:
        md_content += f"- **{dim['name']}**: {dim['score']} - {dim['justification']}\n"
    
    md_content += f"""
## 리뷰
{result_dict.get('review', '리뷰 없음')}

## 관련 연구
"""
    for w in result_dict.get('related_works', []):
        md_content += f"- [{w['title']}](https://arxiv.org/abs/{w['arxiv_id']}) (관련도: {w['relevance']:.2f})\n"
    
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(md_content)
    
    # 최신 결과를 review.json으로도 저장 (노트북 4 비교용)
    latest_json = os.path.join(output_dir, "review.json")
    with open(latest_json, 'w', encoding='utf-8') as f:
        json.dump(result_dict, f, ensure_ascii=False, indent=2)
    
    print(f"💾 결과 저장:")
    print(f"   JSON: {json_path}")
    print(f"   Markdown: {md_path}")
    print(f"   (+ review.json 최신 버전 업데이트)")
    print(f"   리뷰 길이: {len(result_dict['review'])} 문자")
    print(f"   점수: {scores.get('final_score_display', 'N/A')}")
else:
    print("⚠️ 리뷰 결과가 없습니다")

---
## Part 4: 결과 공유 및 토론

### 결과 제출 방법

1. **아래 셀에서 이름 입력**
2. **셀 실행 → 파란 버튼 클릭**
3. **폼에서 의견 작성 후 제출**

> 점수는 자동 입력됩니다!

### 링크

| 용도 | 링크 |
|------|------|
| **직접 제출** (자동 입력 안 될 때) | [Google Forms](https://docs.google.com/forms/d/e/1FAIpQLSfciPtMCZTSNyGvutdFGSdcUjKSdu98Vm7gVPe6TvVcGQKK2g/viewform) |
| **결과 확인** | [Google Sheet](https://docs.google.com/spreadsheets/d/1wPGTOPGF5yvWQTimikr-rg2VExXiHCE0Xn2EVkffWfo/edit?usp=sharing) |

### 토론 질문

1. **AI 리뷰가 유용했나요?** (1-5점)
2. **어떤 점이 유용했나요?**
3. **실제 리뷰와 공통점**은?
4. **실제 리뷰와 차이점**은?
5. **개선이 필요한 부분**은?

In [ ]:
# Cell 9: 결과 제출
# ============================================================
# 이름만 입력하세요! 나머지는 자동으로 채워집니다.
# ============================================================

PARTICIPANT_NAME = ""  # <- 본인 이름 입력

# ============================================================
# 실험 결과 자동 수집
# ============================================================
import urllib.parse
import json
from datetime import datetime
from IPython.display import display, HTML

if not PARTICIPANT_NAME:
    print("⚠️  PARTICIPANT_NAME을 입력하세요!")
elif 'review_result' not in dir() or not review_result:
    print("⚠️  Cell 6을 먼저 실행하세요! (리뷰 실행)")
else:
    # 결과 추출
    scores = review_result.get('scores', {})
    dimensions = scores.get('dimensions', [])
    
    # 점수 추출
    final_score = scores.get('final_score_display', '')
    soundness = ""
    presentation = ""
    contribution = ""
    
    for dim in dimensions:
        name = dim.get('name', '').lower()
        score = str(dim.get('score', ''))
        if 'sound' in name:
            soundness = score
        elif 'present' in name:
            presentation = score
        elif 'contribu' in name:
            contribution = score
    
    # 사용한 논문 유형
    paper_type = "샘플 논문" if "sample_manuscript" in PAPER_PATH else "본인 논문"
    
    # Pre-filled Google Form URL 생성
    FORM_BASE = "https://docs.google.com/forms/d/e/1FAIpQLSfciPtMCZTSNyGvutdFGSdcUjKSdu98Vm7gVPe6TvVcGQKK2g/viewform?usp=pp_url"
    
    params = {
        "entry.1176756138": PARTICIPANT_NAME,  # 이름
        "entry.1543210690": paper_type,        # 사용한 논문
        "entry.538415097": final_score,        # 최종 점수
        "entry.233528916": soundness,          # Soundness
        "entry.2053546740": presentation,      # Presentation
        "entry.1438513897": contribution,      # Contribution
        # 나머지는 폼에서 직접 작성
        # entry.620778037: AI 리뷰 유용성 (1-5)
        # entry.1900039021: 어떤 점이 유용했나요?
        # entry.2048465412: 실제 리뷰와 공통점
        # entry.1192534948: 실제 리뷰와 차이점
        # entry.1803074178: 개선이 필요한 부분
    }
    
    form_url = FORM_BASE + "&" + urllib.parse.urlencode(params)
    
    # 결과 미리보기
    print("=" * 70)
    print("📋 제출 내용 미리보기")
    print("=" * 70)
    print(f"\n👤 이름: {PARTICIPANT_NAME}")
    print(f"📄 논문: {paper_type}")
    print(f"\n🎯 최종 점수: {final_score}")
    print(f"   Soundness: {soundness}")
    print(f"   Presentation: {presentation}")
    print(f"   Contribution: {contribution}")
    
    # 클릭 가능한 링크 출력
    print("\n" + "=" * 70)
    print("🔗 아래 버튼 클릭 → 의견 작성 → 제출")
    print("=" * 70)
    
    display(HTML(f'''
    <a href="{form_url}" target="_blank" 
       style="display:inline-block; font-size:16px; padding:12px 24px; 
              background:#4285f4; color:white; text-decoration:none; 
              border-radius:5px; margin:10px 0;">
       📝 결과 제출하기 (클릭)
    </a>
    <p style="color:#666; font-size:12px;">※ 점수는 자동 입력됩니다. 의견만 작성하세요!</p>
    '''))
    
    # 로컬 백업 저장
    save_data = {
        'participant': PARTICIPANT_NAME,
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'paper_type': paper_type,
        'scores': {
            'final': final_score,
            'soundness': soundness,
            'presentation': presentation,
            'contribution': contribution
        },
        'review': review_result.get('review', '')[:500]  # 처음 500자만
    }
    
    if IN_COLAB:
        save_folder = "/content/drive/MyDrive/Hands-on-3/outputs/3_agent_result/"
    else:
        save_folder = os.path.join(WORKSHOP_DIR, "outputs", "3_agent_result")
    
    os.makedirs(save_folder, exist_ok=True)
    filename = f"{PARTICIPANT_NAME}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    filepath = os.path.join(save_folder, filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(save_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n✓ 로컬 백업: {filepath}")

---
## 정리

### 이번 노트북에서 배운 것

1. **기존 오픈소스 Paper Review Agent 도구들** 탐색
   - agentic-paper-review, AgentReview, AI-Scientist
2. **agentic-paper-review 실행**: Python API로 논문 리뷰
3. **LangGraph 기반 9노드 워크플로우** 이해
   - PDF→MD → 메타데이터 → 쿼리생성 → 웹검색 → 관련성평가 → 리플렉션 → 요약 → 리뷰생성 → 점수산정
4. **다양한 파일 형식 지원**: PDF, MD, TXT, DOCX

### 다음 노트북 예고

- **Notebook 4**: 직접 리뷰 에이전트 구현 & 결과 비교

### 참고 자료

- agentic-paper-review: https://github.com/debashis1983/agentic-paper-review
- AgentReview Paper: https://arxiv.org/abs/2406.12708
- LangGraph Docs: https://python.langchain.com/docs/langgraph